In [1]:
import pandas as pd
import json
from collections import OrderedDict
from itertools import groupby
from datetime import datetime
from pymongo import MongoClient

In [2]:
# Connect
client = MongoClient("mongodb://mongo:27017")

# Connect to `jupyter` database
db = client['jupyter']

In [3]:
# NOTE - change the `nrows=None` argument to a smaller number for tesitng
# i.e. `nrows=100` will only load the first 100 rows
df = pd.read_csv('Food_Service_Establishment_Inspections__Beginning_2005__ACTIVE_.csv', nrows=1000)

In [4]:
df.head()

,FACILITY,ADDRESS,DATE OF INSPECTION,VIOLATION ITEM,VIOLATION DESCRIPTION,CRITICAL VIOLATION,TOTAL # CRITICAL VIOLATIONS,TOTAL #CRIT. NOT CORRECTED,TOTAL # NONCRITICAL VIOLATIONS,LOCAL HEALTH DEPARTMENT,...,PERMITTED (D/B/A),PERMITTED CORP. NAME,PERM. OPERATOR LAST NAME,PERM. OPERATOR FIRST NAME,NYS HEALTH OPERATION ID,INSPECTION TYPE,INSPECTION COMMENTS,FS FACILITY STATE,LATITUDE,LONGITUDE
0,STEVIE V'S - LA ROSA PIZZERIA,"6318 ROBINSON ROAD, LOCKPORT",10/03/2017,15A,"Floors, walls, ceilings, not smooth, properly ...",Not Critical Violation,0.0,0.0,3.0,Niagara County,...,NaN,STEVIE V'S - LA ROSA PIZZERIA,VERBOCY,STEPHEN,286815,Inspection,Restrooms: OK\nFood Testing Thermometers: OK\n...,NY,43.124090,-78.736710
1,SCOOPS-N-MORE,"90 GLENEIDA AVENUE, CARMEL",09/09/2015,10B,Non-food contact surfaces and equipment are im...,Not Critical Violation,0.0,0.0,1.0,Putnam County,...,NaN,NaN,MORRIS,THERESA,299840,Inspection,NaN,NY,41.429351,-73.679427
2,J & B LUNCHEONETTE,"189 ROUTE 9W, HAVERSTRAW",04/15/2010,None,NaN,Not Critical Violation,0.0,0.0,0.0,Rockland County,...,J & B LUNCHEONETTE,"JERRY ACCOMANDO, INC.",ACCOMANDO,JERRY,303561,Inspection,NaN,NY,41.198585,-73.981861
3,STOLTZFUS PASTRIES,"2325 McCULLOCH ROAD, ROMULUS",07/12/2012,None,NaN,Not Critical Violation,0.0,0.0,0.0,Seneca County,...,NaN,NaN,STOLTZFUS,REBECCA,308241,Inspection,NaN,NY,42.786649,-76.809727
4,BOATYARD GRILL,"525 TAUGHANNOCK BOULEVARD, ITHACA",08/26/2011,None,NaN,Not Critical Violation,0.0,0.0,0.0,Tompkins County,...,NaN,THE BOATYARD GRILL,CAMPAGNOLO,MARK,313774,Re-Inspection,NaN,NY,42.442929,-76.513121


In [5]:
# df.shape

In [6]:
# df['TOTAL # NONCRITICAL VIOLATIONS'].mean()

In [7]:
# df.groupby(['LOCAL HEALTH DEPARTMENT'])['TOTAL # NONCRITICAL VIOLATIONS'].mean().sort_values()

In [8]:
names = df.columns.values.tolist()
data = df.values

In [9]:
names

['FACILITY',
 'ADDRESS',
 'DATE OF INSPECTION',
 'VIOLATION ITEM',
 'VIOLATION DESCRIPTION',
 'CRITICAL VIOLATION',
 'TOTAL # CRITICAL VIOLATIONS',
 'TOTAL #CRIT.  NOT CORRECTED ',
 'TOTAL # NONCRITICAL VIOLATIONS',
 'LOCAL HEALTH DEPARTMENT',
 'COUNTY',
 'FACILITY CODE',
 'FACILITY ADDRESS',
 'FACILITY CITY',
 'FACILITY POSTAL ZIPCODE',
 'NYSDOH GAZETTEER (1980)',
 'FACILITY MUNICIPALITY',
 'OPERATION NAME',
 'PERMIT EXPIRATION DATE',
 'FOOD SERVICE TYPE',
 'FOOD SERVICE DESCRIPTION',
 'PERMITTED  (D/B/A)',
 'PERMITTED  CORP. NAME',
 'PERM. OPERATOR LAST NAME',
 'PERM. OPERATOR FIRST NAME',
 'NYS HEALTH OPERATION ID',
 'INSPECTION TYPE',
 'INSPECTION COMMENTS',
 'FS FACILITY STATE',
 'LATITUDE',
 'LONGITUDE']

In [10]:
results = dict()

violationLookup = dict()

facilityAttributes = [
    "NYS HEALTH OPERATION ID",
    "FACILITY",
    "DATE OF INSPECTION",
    "FACILITY POSTAL ZIPCODE",
    "ADDRESS",
    "FOOD SERVICE DESCRIPTION",
    "FOOD SERVICE TYPE",
    "OPERATION NAME",
    "LATITUDE",
    "LONGITUDE",
    "FS FACILITY STATE",
    "INSPECTION TYPE",
    "INSPECTION COMMENTS",
    "PERM. OPERATOR LAST NAME",
    "PERM. OPERATOR FIRST NAME",
    "PERMITTED  CORP. NAME"
]

print('Processing CSV...')

for (
    nysho_id,
    facility,
    date,
    zipcode,
    address,
    desc,
    ftype,
    opname,
    lat,
    long,
    state,
    insp_type,
    insp_comment,
    op_last,
    op_first,
    corp_name
), bag in df.groupby(facilityAttributes):
    
    default = dict({
        'facility_id': int(nysho_id,),
        'facility': facility.strip().title(),
        'operation_name': opname.strip().title(),
        'corp_name': corp_name.strip().title().replace('Llc', 'LLC'),
        'type': ftype,
        'description': desc,
        'operator': dict({
            'first': op_first.strip().title(),
            'last': op_last.strip().title()
        }),
        'address': dict({
            'street': address.split(',  ')[0].strip().title(),
            'city': address.split(',  ')[1].strip().title(),
            'state': state.strip().upper(),
            'zipcode': str(zipcode).strip(),
            'coords': [lat, long]
        }),
        'inspections': []
    })
    
    fac = results.get(facility.strip(), default)
    
    contents_df = bag.drop(facilityAttributes, axis=1)
    
    inspection = dict({
        #'date': datetime.strptime(date, "%m/%d/%y"),
        'date': date,
        'type': insp_type,
        'comment': insp_comment,
        'violations': []
    })
    
    for i, row in contents_df.iterrows():
        inspection['violations'].append(row['VIOLATION ITEM'])
        violationLookup[row['VIOLATION ITEM']] = row['VIOLATION DESCRIPTION']
    
    fac['inspections'].append(inspection)
    results[facility.strip()] = fac

Processing CSV...
Writing output...


In [11]:
# Iterates over each Restaurant and inserts it into Mongo
for k, v in results.items():
    db['restaurants'].insert_one(v)
    # print(json.dumps(v, indent=2)) 

# Logs..
print('Wrote Restaurants to MongoDb...')

Wrote Restaurants to MongoDb...


In [12]:
# Writes the Restaurants to JSON
# file = open('restaurants.json','w') 
# file.write(json.dumps(output, indent=2)) 
# file.close()
# print(json.dumps(violationLookup, indent=2))

In [13]:
# Iterates over each Violation and inserts it into Mongo
for v in violationLookup.items():
    db['violations'].insert_one(dict({ 'vid': v[0], 'desc': v[1]}))
    # print(json.dumps(dict({ 'vid': v[0], 'desc': v[1]}), indent=2))

# Logs..
print('Wrote Violations to MongoDb...')

Wrote Violations to MongoDb...
